

# Loading the Data



In [1]:
import pickle

# Define the path to your data file in Google Drive
file_path_cap = 'cap.pck'
file_path_pref = 'pref.pck'


# Load capacity dictionary
with open(file_path_cap, "rb") as f:
    cap = pickle.load(f)

# Load preference/priority dictionary
with open(file_path_pref, "rb") as f:
    pref = pickle.load(f)

In [2]:
# School IDs directly from cap
school_ids = list(cap.keys())

# Student IDs are in pref but not in cap
student_ids = [k for k in pref.keys() if k not in cap]

print(f"Number of schools: {len(school_ids)}")
print(f"Sample school IDs: {school_ids}")

print(f"Number of students: {len(student_ids)}")
print(f"Sample student IDs: {student_ids[:5]}")

Number of schools: 100
Sample school IDs: ['8412_401000000133_REG', '8412_401000000133_PRI', '8413_401000000133_REG', '8413_401000000133_PRI', '8415_401000000133_REG', '8415_401000000133_PRI', '8417_401000000133_REG', '8417_401000000133_PIE', '8417_401000000133_PRI', '8420_401000000131_REG', '8420_401000000131_PRI', '8421_401000000133_REG', '8421_401000000133_PRI', '8422_401000000112_REG', '8422_401000000112_PRI', '8433_401000000131_REG', '8433_401000000131_PRI', '8433_401000000132_REG', '8433_401000000132_PRI', '8434_401000000133_REG', '8434_401000000133_PRI', '8435_401000000133_REG', '8435_401000000133_PRI', '8436_401000000133_REG', '8436_401000000133_PRI', '8437_401000000133_REG', '8437_401000000133_PIE', '8437_401000000133_PRI', '8438_401000000133_REG', '8438_401000000133_PRI', '8439_401000000131_REG', '8439_401000000131_PRI', '8439_401000000132_REG', '8439_401000000132_PRI', '8441_401000000133_REG', '8441_401000000133_PRI', '8442_401000000133_REG', '8442_401000000133_PRI', '8448_4

# Combining Regular and Priority Schools


In [3]:
from collections import defaultdict

In [4]:
merged_cap = {}
merged_pref = {}

# Identify base school IDs
base_school_ids = set()
for sid in cap.keys():
    if sid.endswith("_REG") or sid.endswith("_PRI"):
        base_id = sid.rsplit("_", 1)[0]
        base_school_ids.add(base_id)

In [5]:
# Merge REG and PRI schools
for base_id in base_school_ids:
    reg_id = base_id + "_REG"
    pri_id = base_id + "_PRI"

    # Capacity merge
    cap_reg = cap.get(reg_id, 0)
    cap_pri = cap.get(pri_id, 0)
    merged_cap[base_id] = cap_reg + cap_pri

    # Get priority lists from dicts sorted by rank (key)
    pri_list = [v for k, v in sorted(pref.get(pri_id, {}).items())]
    reg_list = [v for k, v in sorted(pref.get(reg_id, {}).items())]

    # Merge without duplicates
    seen = set()
    combined_priority = []
    for student in pri_list + reg_list:
        if student not in seen:
            seen.add(student)
            combined_priority.append(student)

    merged_pref[base_id] = combined_priority

In [6]:
# Add unchanged student preferences
for sid in pref:
    if sid not in cap:  # these are student IDs
        merged_pref[sid] = pref[sid]

In [7]:
print(f"Merged school count: {len(merged_cap)}")
sample_id = list(merged_cap.keys())[0]
print(f"Sample school ID: {sample_id}")
print(f"Capacity: {merged_cap[sample_id]}")
print(f"Top 5 priorities: {merged_pref[sample_id][:5]}")

Merged school count: 49
Sample school ID: 11687_401000000133
Capacity: 26
Top 5 priorities: ['5b86316c093da070f210b297', '5b863166093da070f2109788', '5b863190093da070f2115d9e', '5b8630fa093da070f20ea4d8', '5b863107093da070f20ee113']


In [8]:
import numpy as np
import pandas as pd

# 1. Number of students
num_students = len([k for k in pref if k not in cap])

# 2. Number of schools
num_schools = len(merged_cap)

# 3. Average number of schools ranked per student
student_prefs = {k: v for k, v in pref.items() if k not in cap}
avg_schools_ranked = np.mean([len(v) for v in student_prefs.values()])

# 4. School capacity stats
school_capacities = list(merged_cap.values())
min_capacity = min(school_capacities)
max_capacity = max(school_capacities)
total_capacity = sum(school_capacities)

# 5. Table summary as a DataFrame
summary_df = pd.DataFrame({
    "Statistic": [
        "Number of students",
        "Number of schools",
        "Average number of schools ranked per student",
        "Minimum school capacity",
        "Maximum school capacity",
        "Total original capacity (all schools)",
        "PIE students included",
        "School preference list merge rule"
    ],
    "Value": [
        num_students,
        num_schools,
        round(avg_schools_ranked, 2),
        min_capacity,
        max_capacity,
        total_capacity,
        "No (excluded)",
        "Priority ranked above Regular"
    ]
})

In [9]:
print(summary_df)

                                      Statistic                          Value
0                            Number of students                           1395
1                             Number of schools                             49
2  Average number of schools ranked per student                           6.47
3                       Minimum school capacity                              3
4                       Maximum school capacity                             80
5         Total original capacity (all schools)                           1620
6                         PIE students included                  No (excluded)
7             School preference list merge rule  Priority ranked above Regular


In [44]:
import gurobipy as gp
from gurobipy import GRB

# Utility: remove seat-level suffix (e.g., "_REG", "_PRI", "_PIE")
def normalize_school_id(raw_id):
    return raw_id.rsplit('_', 1)[0]

# Utility: remove duplicates from a list while preserving order
def dedup(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

# Step 1: Build ordered and normalized student preferences
ordered_student_prefs = {}
for s, p in student_prefs.items():
    if isinstance(p, dict):
        ordered_raw = [v for _, v in sorted(p.items())]
    elif isinstance(p, list):
        ordered_raw = p
    else:
        continue

    normalized = [normalize_school_id(school_id) for school_id in ordered_raw]
    ordered_student_prefs[s] = dedup(normalized)

# Step 2: Build ordered school preferences (already merged)
ordered_school_prefs = {}
for c, p in merged_pref.items():
    if isinstance(p, dict):
        ordered_school_prefs[c] = [v for _, v in sorted(p.items())]
    else:
        ordered_school_prefs[c] = p

# Step 3: Define feasible student-school pairs (F)
# Allow: any school that student ranked, and exists in capacity list
C = list(merged_cap.keys())
S = list(ordered_student_prefs.keys())

F = [(s, c) for s in S for c in ordered_student_prefs[s] if c in C]
print(f"Feasible (student, school) pairs: {len(F)}")

# Recompute utility weights: normalized linearly from student preference rank
u = {}
for s, prefs in ordered_student_prefs.items():
    L = len(prefs)
    for rank, c in enumerate(prefs):
        u[(s, c)] = 1 - (rank / L)  # Top choice gets 1, last gets 1/L
# Capacities and budget
q = merged_cap
B = 100  # example budget


Feasible (student, school) pairs: 4493


In [45]:
m = gp.Model("QuadraticStableMatching")

# Decision variables
x = m.addVars(F, vtype=GRB.BINARY, name="x")
t = m.addVars(C, vtype=GRB.INTEGER, lb=0, name="t")

# Objective: Maximize total utility
m.setObjective(gp.quicksum(u[s, c] * x[s, c] for (s, c) in F), GRB.MAXIMIZE)

# Constraints
for s in S:
    m.addConstr(gp.quicksum(x[s, c] for c in C if (s, c) in F) <= 1)

for c in C:
    m.addConstr(gp.quicksum(x[s, c] for s in S if (s, c) in F) <= q[c] + t[c])

m.addConstr(gp.quicksum(t[c] for c in C) <= B)

# Quadratic Stability Constraints
for s, c in F:
    if c not in ordered_student_prefs[s]: continue  # skip infeasible match
    
    student_rank = ordered_student_prefs[s].index(c)
    better_schools = ordered_student_prefs[s][:student_rank]

    if c not in ordered_school_prefs: continue
    if s not in ordered_school_prefs[c]: continue

    school_rank = ordered_school_prefs[c].index(s)
    better_students = ordered_school_prefs[c][:school_rank]

    m.addQConstr(
        (q[c] + t[c]) * x[s, c] +
        (q[c] + t[c]) * gp.quicksum(x[s, c2] for c2 in better_schools if (s, c2) in F) +
        gp.quicksum(x[s2, c] for s2 in better_students if (s2, c) in F) >= 1
    )


In [46]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1445 rows, 4542 columns and 9084 nonzeros
Model fingerprint: 0x5ec3a540
Model has 4493 quadratic constraints
Variable types: 0 continuous, 4542 integer (4493 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 8e+01]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
  QRHS range       [1e+00, 1e+00]
Presolve removed 106 rows and 231 columns
Presolve time: 0.12s
Presolved: 7189 rows, 6866 columns, 66541 nonzeros
Variable types: 0 continuous, 6866 integer (4272 binary)
Found heuristic solution: objective 1114.8662990

Root relaxation: objective 1.256817e+03, 371 iterations, 0.01 seconds (0.01 work un

In [47]:
m.update()  # not strictly necessary, but safe
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")


Model has 4542 variables and 1445 constraints.


In [48]:
if m.Status == GRB.OPTIMAL:
    matches = [(s, c) for (s, c) in F if x[s, c].X > 0.5]
    print(f"Number of matches: {len(matches)}")
else:
    print("Model did not solve to optimality.")

Number of matches: 1394


In [49]:
num_matched = len(set(s for (s, c) in matches))
print(f"Total students matched: {num_matched} out of {len(S)}")

Total students matched: 1394 out of 1395


In [50]:
total_utility = sum(u[s, c] for (s, c) in matches)
avg_utility = total_utility / num_matched if num_matched > 0 else 0
print(f"Average student utility: {avg_utility:.3f}")


Average student utility: 0.902


In [51]:
capacity_used = {c: sum(1 for (s2, c2) in matches if c2 == c) for c in C}
capacity_expanded = {c: int(t[c].X) for c in C}
capacity_original = {c: q[c] for c in C}

print("\nSchool Capacity Overview:")
for c in C:
    print(f"School {c}: Used = {capacity_used.get(c,0)}, "
          f"Original = {capacity_original[c]}, "
          f"Added = {capacity_expanded[c]}")



School Capacity Overview:
School 11687_401000000133: Used = 7, Original = 26, Added = 0
School 8435_401000000133: Used = 14, Original = 30, Added = 0
School 8448_401000000133: Used = 12, Original = 15, Added = 0
School 24302_401000000131: Used = 8, Original = 22, Added = 0
School 8433_401000000131: Used = 19, Original = 50, Added = 0
School 8412_401000000133: Used = 60, Original = 60, Added = 0
School 8454_401000000222: Used = 80, Original = 80, Added = 0
School 24305_401000000332: Used = 38, Original = 33, Added = 5
School 8441_401000000133: Used = 29, Original = 60, Added = 0
School 24313_401000000131: Used = 31, Original = 31, Added = 0
School 8438_401000000133: Used = 35, Original = 48, Added = 0
School 8437_401000000133: Used = 11, Original = 28, Added = 0
School 8439_401000000132: Used = 30, Original = 30, Added = 0
School 8449_401000000133: Used = 6, Original = 30, Added = 0
School 24329_401000000122: Used = 33, Original = 35, Added = 0
School 8467_401000000133: Used = 0, Origi